In [ ]:
import os
import re
import json
import chardet
import src.public_function as pf

# NOTE 中间的所有文件都是UTF8
encoding = 'utf16'
output_encoding = 'utf16'

pf._init_()
name = ''

In [ ]:
# 从游戏原文本文件提取日文

def get_scenario_from_origin(data: list) -> list:
    text_all = []
    cnt = 0
    # START FIXME
    for line in data:
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            text_all.append(line)
        cnt+=1
    # END
    return text_all

pf.extract_jp(get_scenario_from_origin, encoding)


In [ ]:
# 创建翻译字典
pf.create_dict(lambda x:x[:-1])

In [ ]:
# 检查字典key数量是否正确
pf.check_dict()

In [ ]:
# 把字典翻译

# print(func2('「・・・ああ、うん、それで・・・スペシャル餃子、一つ・・・」'))
pf.translate()
# print(pf._translate('＄β,はい、＄αです……あっ、先輩！　すみません、僕のせいでシフトが大幅に変わっちゃって……ご迷惑をおかけして本当にすみません', delete_func=lambda x: pf.PAC.format_pac(x, {'＄β':'优登','＄α':'天海'})))

In [ ]:
# 检查是否还有没翻译的语句
pf.check_dict_untranslated()

In [ ]:
# 构建对比
pf.create_contrast()

In [ ]:
# 替换文本
def find(data:list, failed_text:list, jp_chs:dict):
    record = 0
    cnt = 0                         # 当前行
    start = False
    for line in data:
        #START FIXME 替换目标文本
        if pf.has_jp(line) and line[0] not in ';[/*\t# ':
            key = line[:-1]
            if key in jp_chs and jp_chs[key]:
                record += 1
                data[cnt] = data[cnt].replace(key, jp_chs[key])
            else:
                failed_text.append(key+'\n')
        cnt += 1
        # END 
    return record

pf.replace_all(find, encoding, output_encoding)

In [ ]:
jp_chs = dict()
chs_file = os.listdir('chs')
for f in chs_file:
    _chs = pf.open_json(f'chs/{f}')
    f = f.split('.')[0]+'.json'
    json_root = 'intermediate_file/split_scenario/'+f
    if os.path.exists(json_root):
        _jp = pf.open_json(json_root)
        _key_chs = list(_chs.keys())
        _key_jp = list(_jp.keys())
        if len(_key_chs) == len(_key_jp):
            for i in range(len(_key_jp)):
                jp_chs[_key_jp[i]] = _key_chs
            # print('输入成功：', f)
        else:
            print('中日不匹配：', f)
    else:
        print('文件不存在：', f)
pf.save_json('intermediate_file/jp_chs.json', jp_chs)